In [1]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [2]:
INPUT_FOLDER = './sample_images/'
patients = os.listdir(INPUT_FOLDER)
patients.sort()

In [80]:
labels = pd.read_csv('stage1_labels.csv')

In [82]:
id = labels["id"].tolist()
cancer = labels["cancer"].tolist()

In [97]:
y_data = []
for target in patients:
    if target in id:
        y_data.append(cancer[id.index(target)])
    else:
        print(target)
        continue
        

In [141]:
cancer_class = [[1,0],[0,1]]
y_labels = []
for i in y_data:
    y_labels.append(cancer_class[i])    

In [3]:
# Load the scans in given folder path
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [5]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [9]:
first_patient = load_scan(INPUT_FOLDER + patients[0])
first_patient_pixels = get_pixels_hu(first_patient)
# plt.hist(first_patient_pixels.flatten(), bins=80, color='c')
# plt.xlabel("Hounsfield Units (HU)")
# plt.ylabel("Frequency")
# plt.show()

# # Show some slice in the middle
# plt.imshow(first_patient_pixels[80], cmap=plt.cm.gray)
# plt.show()

In [28]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + scan[0].PixelSpacing, dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [103]:
input_data = []
for i in patients:
    patient = load_scan(INPUT_FOLDER + i)
    patient_pixels = get_pixels_hu(patient)
    input_data.append(patient_pixels)

In [111]:
input_data = np.array(input_data)

In [11]:
MIN_BOUND = -1000.0
MAX_BOUND = 400.0
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

In [12]:
PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

In [118]:
rectified_data = []
for row in input_data:
    norm = normalize(row)
    zero = zero_center(norm)
    rectified_data.append(zero)

In [165]:
def get_onehot(data):
    onehot_data = []
    for person in range(len(data)):
        temp = []
        for pic in data[person]:
            temp1 = pic.reshape(-1)
            temp.append(np.array(temp1))
        onehot_data.append(np.array(temp))
    return np.array(onehot_data)

In [166]:
onehot = get_onehot(rectified_data)

In [176]:
onehot

array([ array([[-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       ..., 
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25]]),
       array([[-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       ..., 
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25]]),
       array([[-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       ..., 
       [-0.25, -0.25, -0.25, ..., -0.25, -0.25, -0.25],
       [-0.25, -0.25, -0.25, ..., -0.25

In [168]:
np.array(onehot[0]).dtype

dtype('float64')

In [13]:
import tensorflow as tf

In [169]:
x = tf.placeholder(tf.float64, [None, 512*512])
input_x = tf.reshape(x, [-1, 512, 512, 3])
input_y = tf.placeholder(tf.float64, [None, 2])

In [146]:
filter_w = tf.Variable(tf.truncated_normal([5, 5, 3, 3], stddev = 0.1))
filter_b = tf.Variable(tf.truncated_normal([3]))

In [154]:
conv = tf.nn.conv2d(input_x, filter_w, strides=[1, 2, 2, 1], padding="VALID")

In [155]:
h_1 = tf.nn.relu(tf.nn.bias_add(conv, filter_b))

In [156]:
pool_1 = tf.nn.max_pool(h_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [160]:
pool_flat = tf.reshape(pool_1, [-1, 127*127*3])

In [158]:
W_fc = tf.Variable(tf.random_normal([127*127*3, 2], stddev=0.1))
b_fc = tf.Variable(tf.random_normal([2], stddev=0.1))

In [162]:
hypothesis = tf.nn.softmax(tf.matmul(pool_flat, W_fc) + b_fc)
cost = -tf.reduce_sum(input_y * tf.log(tf.clip_by_value(hypothesis,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.01, epsilon=0.05).minimize(cost)

In [163]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [175]:
for i in range(10001):
    sess.run(train_step, feed_dict={x: onehot, input_y: y_labels})
    if i % 1000 == 0:
        print(i, sess.run(cost, feed_dict={x: onehot, input_y: y_labels}))

ValueError: setting an array element with a sequence.